<a href="https://colab.research.google.com/github/KodakV/train_datasets/blob/master/ClassificationWine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import random
import numpy as np

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True


In [ ]:
import sklearn.datasets
wine = sklearn.datasets.load_wine()
wine.data.shape

(178, 13)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    wine.data[:, :2], 
    wine.target, 
    test_size=0.3, 
    shuffle=True)
print(X_train.shape)
print(y_train.shape)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

(124, 2)
(124,)


In [ ]:
class WineNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(WineNet, self).__init__()
        
        self.fc1 = torch.nn.Linear(2, n_hidden_neurons)
        self.activ1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, n_hidden_neurons)
        self.activ2 = torch.nn.Sigmoid()
        self.fc3 = torch.nn.Linear(n_hidden_neurons, 3)
        self.sm = torch.nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activ1(x)
        x = self.fc2(x)
        x = self.activ2(x)
        x = self.fc3(x)
        return x

    def inference(self, x):
        x = self.forward(x)
        x = self.sm(x)
        return x
    
wine_net = WineNet(5)

In [ ]:
loss = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(wine_net.parameters(), 
                             lr=1.0e-3)

In [ ]:
print(type(y_train))

<class 'torch.Tensor'>


In [ ]:
np.random.permutation(5)

array([2, 0, 1, 4, 3])

In [ ]:
batch_size = 10

for epoch in range(5000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        x_batch = X_train[batch_indexes]
        y_batch = y_train[batch_indexes]
        
        preds = wine_net.forward(x_batch) 
        print(x_batch.size() ,y_batch.size(), preds.size())
        loss_value = loss(preds, y_batch)
        loss_value.backward()
        
        optimizer.step()
        
    if epoch % 100 == 0:
        test_preds = wine_net.forward(X_test)
        test_preds = test_preds.argmax(dim=1)
        print((test_preds == y_test).float().mean())

Выходные данные были обрезаны до нескольких последних строк (5000).
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([4, 2]) torch.Size([4]) torch.Size([4, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([10, 2]) torch.Size([10]) torch.Size([10, 3])
torch.Size([4, 2]) torch.Size([4]) torch.Size([4, 3])
torch.Size([10, 2]) torch.

KeyboardInterrupt: ignored